In [1]:
import pandas as pd
import numpy as np
# import random as rn
# import sklearn
# from scipy import stats
# import math
import re

from sklearn.preprocessing import  LabelEncoder #OneHotEncoder
# from sklearn import ensemble
from sklearn import linear_model

# from sklearn.model_selection import GridSearchCV,cross_val_score
# from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,accuracy_score

# # from sklearn.ensemble import RandomForestClassifier
# from sklearn import linear_model

# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV

import pickle

In [11]:
totDF = pd.read_csv('../../data/processed/Cleaned_Data_Set.csv')

In [12]:
def cleanDF (df):
    r1 = re.compile('.*reporting')
    r2 = re.compile('.*imputed')

    cols_to_drop1 = list(filter((r1.match), df.columns))
    cols_to_drop2 = list(filter((r2.match), df.columns))
    cols_to_drop3 = ['admit_NICU']
    cols_to_drop = cols_to_drop1 + cols_to_drop2 + cols_to_drop3

    cols_to_keep = [col for col in df.columns if col not in cols_to_drop]

    X_and_target = df[cols_to_keep + ['admit_NICU']].copy()

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    catDF = X_and_target.select_dtypes(include=object).copy()
    numDF = X_and_target.select_dtypes(include=numerics).copy() #only numeric columns

    le = LabelEncoder()
    catDF = catDF.apply(le.fit_transform)

    concat_df = pd.concat([numDF,catDF],axis=1)
    return concat_df

In [37]:
cl_df = cleanDF(totDF)
nicu_allY = cl_df.loc[cl_df['admit_NICU']==2]
nicu_allN = cl_df.loc[cl_df['admit_NICU']==0]
cl_df_X = cl_df.drop('admit_NICU',axis=1)

In [38]:
nicu_YN = pd.concat([nicu_allY, nicu_allN],axis=0)
nicu_YN_X = nicu_YN.drop('admit_NICU', axis =1)

In [3]:
bal100K_model = pickle.load(open('bal100K_model.sav', 'rb'))

In [39]:
y_pred = bal100K_model.predict(nicu_YN_X)

In [49]:
bal100K_model.best_score_

0.84814

In [41]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(nicu_YN.admit_NICU,y_pred)
np.set_printoptions(suppress=True)
print(cf/len(cl_df)*100)

[[80.6777  9.4698]
 [ 1.7066  6.9375]]
